##### 问题:
我们已经编写好了一个装饰器，但是当将它用在一个函数上时，一些重要的元数据比
如函数名、文档字符串、函数注解以及调用签名都丢失了。

##### 解决方案；
每当定义一个装饰器时，应该总是记得为底层的包装函数添加 functools 库中的@wraps
装饰器。示例如下：

In [65]:
import time
from functools import wraps
def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper 

In [66]:
@timethis
def countdown(n:int):
    while n > 0:
        n -= 1

In [67]:
countdown(100000) 
print(countdown.__name__)
print(countdown.__doc__)
print(countdown.__annotations__)


countdown 0.004986763000488281
countdown
None
{'n': <class 'int'>}


编写装饰器的一个重要部分就是拷贝装饰器的元数据。如果忘记使用@wraps，就会发
现被包装的函数丢失了所有有用的信息。例如，如果忽略@wraps，上面这个例子中的
元数据看起来就是这样的：

In [68]:
import time
from functools import wraps
def timethis(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper
@timethis
def countdown(n:int):
    while n > 0:
        n-= 1 
countdown(100000) 
print(countdown.__name__)
print(countdown.__doc__)
print(countdown.__annotations__)


countdown 0.004980564117431641
wrapper
None
{}


@wraps 装饰器的一个重要特性就是它可以通过__wrapped__属性来访问被包装的那个
函数。例如，如果希望直接访问被包装的函数，则可以这样做：

In [69]:
import time
from functools import wraps
def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper
@timethis
def countdown(n:int):
    while n > 0:
        n-= 1 
countdown(1)
countdown.__wrapped__(100000)

countdown 0.0


__wrapped__属性的存在同样使得装饰器函数可以合适地将底层被包装函数的签名暴
露出来。例如：(函数签名指的是函数的传入参数的类型)

In [70]:
from inspect import signature
print(signature(countdown))

(n: int)


常会提到的一个问题是如何让装饰器直接拷贝被包装的原始函数的调用签名（即，不
使用*args 和**kwargs）。一般来说，如果不采用涉及生成代码字符串和 exec()的技巧，那
么这很难实现。坦白地说，通常我们最好还是使用@wraps。这样可以依赖于一个事实，
即，底层的函数签名可以通过访问__wrapped__属性来传递。有关函数签名的更多信息
可以参阅 9.16 节。